In [1]:
import osmium
import copy


class Road_info(osmium.SimpleHandler):
    def __init__(self):
        super(Road_info, self).__init__()
        self.ways = {'maxspeed': [], 'lanes': [], 'id': [], 'road_type': []}
        self.nodes = {'loc': [], 'traffic_signals': [], 'id': []}

    def node(self, n):
        tags = n.tags
        if 'highway' in tags:
            traffic_signals = copy.deepcopy(
                tags.get('highway') == 'traffic_signals')
            loc = copy.copy((n.location.lat, n.location.lon))
            id = n.id
            self.nodes['traffic_signals'].append(traffic_signals)
            self.nodes['loc'].append(loc)
            self.nodes['id'].append(id)

    def check(self, name, tags):
        var = None
        if name in tags:
            var = tags[name]
        return var

    def way(self, w):
        tags = w.tags

        if 'highway' in tags:
            road_type = tags.get('highway')
            maxspeed = self.check('maxspeed', tags)
            lanes = self.check('lanes', tags)
            self.ways['maxspeed'].append(maxspeed)
            self.ways['lanes'].append(lanes)
            self.ways['road_type'].append(road_type)
            ids = []
            for node in w.nodes:
                ids.append(node.ref)
            self.ways['id'].append(ids)


In [2]:
handler = Road_info()
path = './data/moscow.osm.pbf'
handler.apply(path)

In [3]:
import pandas as pd

ways = pd.DataFrame(handler.ways)
nodes = pd.DataFrame(handler.nodes).set_index('id')
ways = ways.explode(column='id', ignore_index=True).set_index('id')

In [4]:
dataset = pd.merge(ways, nodes, right_index= True, left_index=True)

In [5]:
dataset = dataset[~dataset.index.duplicated()]
dataset.describe()

,maxspeed,lanes,road_type,loc,traffic_signals
count,16001,25396,54679,54679,54679
unique,24,9,26,54677,2
top,RU:urban,2,tertiary,"(55.3226741, 36.7490176)",False
freq,4962,14810,11638,2,48208


In [6]:
dataset.to_csv('./road_info.csv')